## Creating a machine learning decision tree from scratch

In [18]:
#Importing numpy and pandas
import numpy as np
import pandas as pd

In [19]:
#Accesing the data
!wget -O data.csv https://ml-decision-tree-data.s3.amazonaws.com/data.csv

--2022-10-03 23:47:33--  https://ml-decision-tree-data.s3.amazonaws.com/data.csv
Resolving ml-decision-tree-data.s3.amazonaws.com (ml-decision-tree-data.s3.amazonaws.com)... 52.217.169.57
Connecting to ml-decision-tree-data.s3.amazonaws.com (ml-decision-tree-data.s3.amazonaws.com)|52.217.169.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2742 (2.7K) [text/csv]
Saving to: ‘data.csv’

data.csv            100%[===================>]   2.68K  --.-KB/s    in 0s      

2022-10-03 23:47:34 (31.8 MB/s) - ‘data.csv’ saved [2742/2742]



In [20]:
#Reading the data to a dataframe
data = pd.read_csv('data.csv')
data.head()

,feature1,feature2,feature3,feature4,class
0,5.0,3.5,1.3,0.3,0
1,6.9,3.1,4.9,1.5,1
2,5.8,2.6,4.0,1.2,1
3,6.7,3.0,5.2,2.3,2
4,5.1,3.3,1.7,0.5,0


In [9]:
class Node:
    
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        # Start code here
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value
    # End code here

In [10]:
class DecisionTreeClassifier:
    def __init__(self, min_samples_split=2, max_depth=2):
        # Initialize the root of the decision tree to traverse through the decision tree to None
        self.root = None
        # initialize the stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    def build_tree(self, dataset, curr_depth=0):
        """
        This will be a recursive function to build the decision tree.
        dataset: The data that you will be using for your assignment
        curr_depth: Current depth of the tree
        Returns the leaf node
        """
        # Separate the features and targets into two variables X and Y
        X = dataset[:,:-1]
        Y = dataset[:,-1]
        
        # Extract the number of samples and number of features
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                
                return Node(best_split["feature_index"],  
                            best_split["threshold"], 
                            left_subtree, right_subtree,
                            best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        
        # return the leaf node
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        """
        Function to find out the best split
        dataset: input data
        num_samples: Number of samples present in the dataset
        num_features: Number of features in the dataset
        Returns the best split
        """

        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features in the data
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left ,dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:,-1], dataset_left[:,-1], dataset_right[:,-1]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "entropy")
                    
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        return best_split

    def split(self, dataset, feature_index, threshold):
        """
        Function to split the data to the left child and right child in the decision tree
        dataset: input data
        feature_index: feature index used to locate the index of the feature in a particular row in the dataset
        threshold: threshold value based on which the split will be calculated
        Returns the left and right datavalues from the dataset
        """
        # Hint: Use list comprehension to distinguish which values would be present in left and right
        # subtree on the basis of threshold
        dataset_left = np.array([row for row in dataset if row[feature_index]<= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]> threshold])
        return dataset_left, dataset_right

    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        """
        Function to calculate information gain. This function subtracts the combined information
        of the child node from the parent node.
        parent: value of parent node
        l_child: value of left child node
        r_child: value of right child node
        mode: based on which information gain will be calculated either entropy/gini index
        Returns the information gain
        """
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="entropy":
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        else:
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))

            
        return gain

    def entropy(self, y):
        """
        Function to calculate the entropy
        y: target labels
        Returns entropy
        """
        target_labels = np.unique(y)
        entropy = 0
        for label in target_labels:
            p_label = len(y[y == label]) / len(y)
            entropy += -p_label * np.log2(p_label)
        return entropy

    def gini_index(self, y):
        """
        Function to calculate gini index
        y: target labels
        Returns gini index
        """
        target_labels = np.unique(y)
        gini = 0
        for label in target_labels:
            p_label = len(y[y == label]) / len(y)
            gini += p_label**2
        return 1 - gini

    def calculate_leaf_value(self, Y):
        """
        Function to compute the value of leaf node.Return the most occuring 
        element in Y. Hint: you can use lists
        Y: target labels
        Returns leaf node value
        """
        # 
        Y = list(Y)
        return max(Y, key=Y.count) 

    def print_tree(self, tree=None, indent=" "):
        """
        Function to print the tree. Use the pre-order traversal method to print the decision tree.
        # Do not make any changes in this function
        """

        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X " + str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end=" ")
            self.print_tree(tree.left, indent + indent)
            print("%sright" % (indent), end=" ")
            self.print_tree(tree.right, indent + indent)

    def fit(self, X, Y):
        """
        Function to train the tree.
        X: Features
        Y: Target
        """
        # Concatenate X, Y to create the dataset and call the build_tree function recursively
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def predict(self, X):
        """
        Prediction function to calculate the all the predictions of the matrix of features
        provided using make_predictions function
        X: Matrix of features
        Returns predictions
        """
        predictions = [self.make_predictions(x, self.root) for x in X]
        return predictions

    def make_predictions(self, x, tree):
        """
        Function to predict a single datapoint
        """
        # return the value if the node is a leaf node
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_predictions(x, tree.left)
        else:
            return self.make_predictions(x, tree.right)
        

In [11]:
#Splitting the data manually into 70% train, 30% test
split = np.random.rand(len(data)) < 0.7
training_data = data[split]
testing_data = data[~split]

X_train = training_data[['feature1', 'feature2', 'feature3', 'feature4']].values
X_test = testing_data[['feature1', 'feature2', 'feature3', 'feature4']].values
Y_train = training_data ["class"]
Y_test = testing_data["class"]

#Creating the 
scratch_classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)

y = Y_train.to_numpy().reshape(-1,1)

#Fit the training date
scratch_classifier.fit(X_train,y)

#Print the decisoon tree
scratch_classifier.print_tree()

X 2 <= 1.9 ? 0.911553273406725
 left: 0.0
 right X 2 <= 4.7 ? 0.6373803478944154
  left: X 0 <= 4.9 ? 0.205592508185083
    left: 2.0
    right 1.0
  right X 2 <= 5.0 ? 0.19635725894022443
    left: X 0 <= 6.3 ? 0.31668908831502096
        left: 2.0
        right 1.0
    right 2.0


In [12]:
y_pred = scratch_classifier.predict(X_test)
y_pred=np.array(y_pred).astype('int')
print(y_pred)

[1 1 2 2 0 2 0 1 1 0 1 1 0 1 0 2 0 2 0 1 2 2 0 2 2 1 0 0 0 0 0 1 1 2 2 2 2
 2 1 0 1 2 0 0 1 0 2 2 2]


In [13]:
print('Test Accuracy: %.1f %%' % (np.mean(y_pred == Y_test) * 100))

Test Accuracy: 95.9 %
